In [5]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import csv
from scipy.optimize import minimize 
from scipy import integrate
from datetime import datetime


In [6]:
def L_final(population,population_density,electricity_consumption,GDP):
    a=0.16
    b=0.08
    c=0.02
    k=3.42
    bracket_factor=[1,1.1,1.3,1.5,1.8,2.5]
    bracket_popuation=[5,20,50,100,500,1000]
    bracket=0
    for i in range(len(bracket_factor)-1):
        if population>bracket_popuation[i]:
            bracket+=1
        else:
            break
    return max(0.1,k*bracket_factor[bracket]*(electricity_consumption**(a)*GDP**(b)*population**(c/(1+200/population_density))))
dataframe=pd.read_excel("mcm_city_data.xlsx")
dataset=dataframe.values.tolist()
num=len(dataset)
my_map={}
for i in range(num):
    my_map[dataset[i][0]]=L_final(dataset[i][5],dataset[i][6],dataset[i][4],dataset[i][8])
group_proportion=[0.12,0.19,0.19,0.22,0.17,0.1]
group_wavelength=[425,475,525,575,625,675]


In [9]:
def L_point(L0,r,R):
    if r<R/2:
        return 16/5*L0
    else:
        return 16/5*L0*np.exp(-9.53/R*(r-R/2))
def L_up(L0,R,r0,psi,P,Q):
    return r0*L_point(L0,r0,R)*(24/((np.pi)**3)*P*psi**(2)+1.4*(1-P)*Q*np.cos(psi))
def Li_reflect_t(K_a,K_o,K_n,L0,R,d,i,altitude,Q,P,r,theta,psi,r0,theta0):
    OT=np.array([d-r0*np.cos(theta0),-r0*np.sin(theta0),0])
    OA=np.array([r*np.sin(psi)*np.cos(theta),r*np.sin(psi)*np.sin(theta),r*np.cos(psi)])
    TA=OA-OT
    cos_alpha=np.dot(OA,TA)/(np.linalg.norm(OA)*np.linalg.norm(TA))
    d1_sq=r**2
    d2_sq=np.dot(TA,TA)
    h=r*np.cos(psi)
    N_a=(0.6*10**2)*np.exp((-altitude-h)/(1000))
    N_n=2.04*10**(7-1.13*10**(-4)*(h+altitude))
    N_o=0.51*10**(7-1.3*10**(-4)*(h+altitude))
    sigma_a=K_a*(cos_alpha+0.5*(3*cos_alpha**(2)+1))/(d1_sq+d2_sq)
    sigma_o=K_o*8*np.pi**4*(cos_alpha**2+1)/((d1_sq+d2_sq)*group_wavelength[i]**4)
    sigma_n=sigma_o/K_o*K_n
    out1=L_up(L0,R,r0,psi,P,Q)
    A=sigma_a*N_a
    O=sigma_o*N_o
    N=sigma_n*N_n
    return out1*(A+O+N)
def city_impact_t(K_a,K_o,K_n,Q_a,Q_s,Q_w,i,name,is_coastal,month_snow,altitude,area,distance):
    L0=group_proportion[i]*my_map[name]
    R=math.sqrt(area/np.pi)*1000
    Q= Q_w if is_coastal else Q_a
    Q+=month_snow/12*Q_s
    P=0.4
    #use random method
    sample_size=10
    core_impact=[]
    sur_impact=[]
    #bounds for integration and options
    def limit_theta(psi,r):
        return [0,np.pi*2]
    def limit_psi(r):    
        return [0.1,np.pi/2-0.1]
    def limit_r():
        return [2000,10000]
    options={"epsabs":20,"limit":3}
    for p in range(sample_size):
        r0=np.random.uniform(0,R)
        theta0=np.random.uniform(0,2*np.pi)
        def target_t(theta,psi,r):
            return Li_reflect_t(K_a,K_o,K_n,L0,R,distance*1000,i,altitude,Q,P,r,theta,psi,r0,theta0)
        if (r0<R/2):
            core_impact.append(integrate.nquad(target_t,[limit_theta,limit_psi,limit_r],opts=options)[0])
        else:
            sur_impact.append(integrate.nquad(target_t,[limit_theta,limit_psi,limit_r],opts=options)[0])
    return (sum(core_impact)+sum(sur_impact))/sample_size


In [14]:
dataframe2=pd.read_excel("mcm_impact_data.xlsx")
trainingdata=dataframe2.values.tolist()
sampnum=len(trainingdata)
def loss_function(vector):
    K_a=vector[0]
    K_o=vector[1]
    K_n=vector[2]
    Q_a=vector[3]
    Q_s=vector[4]
    Q_w=vector[5]
    loss=0
    for n in range(sampnum):
        print("currently working on"+trainingdata[n][0])
        guess=0
        for j in range(len(group_proportion)):
            guess+=city_impact_t(K_a,K_o,K_n,Q_a,Q_s,Q_w,j,trainingdata[n][0],trainingdata[n][1],trainingdata[n][2],trainingdata[n][3],trainingdata[n][4],trainingdata[n][5])
#         print("Our prediction:"+str(guess))
        answer=trainingdata[n][6]
#         print("Answer:"+str(answer))
        loss+=(guess-answer)**(2)/(min(guess,answer))
            
    return loss
boundary1=(0,1)
boundary2=(0,3)
my_boundaries=[boundary2,boundary2,boundary2,boundary1,boundary1,boundary2]
final=minimize(loss_function,[0.2,0.2,0.2,0.3,1.7,0.6],bounds=my_boundaries,tol=10)


currently working onNew York
currently working onNew York


KeyboardInterrupt: 